In [23]:
import pandas as pd, re
import numpy as np
import getschools as gs

In [24]:
THE_YEAR="2017"
THE_FYEAR="2016-17"

In [40]:
fyear = lambda x: str(int(x)-1) + "-" + str(int(x) - 2000)
fyear("2017")

'2016-17'

In [25]:
def get_df(group,year):
    ret = pd.read_csv("../scripts/noequals/CTSchoolDaySAT-"+group+"-" + year +".csv",skiprows=2,header=None)
    ret = gs.merge_01(ret)
    ret = gs.clean_colnames(ret)
    
    ret["district_code"] = ret["district_code"].str.replace("\"","")
    ret["school_code"] = ret["school_code"].str.replace("\"","")
    
    ret["year"] = fyear(year)
    
    ret = gs.add_drg(ret)
    ret["average_score"] = ret["level_3_4met_or_exceeded_averagescore"]
    return ret

get_df("all","2017")

,district,district_code,school,school_code,subject,total_numberof_students,total_numbertested,connecticut_school_day_satparticipationrate,total_numberwith_scored_tests,level_1not_met_count,...,level_3met_count,level_3met_pct,level_4exceeded_count,level_4exceeded_pct,level_3_4met_or_exceeded_count,level_3_4met_or_exceeded_pct,level_3_4met_or_exceeded_averagescore,year,drg,average_score
2,Achievement First Hartford Academy Inc. District,2880013,Achievement First Hartford Academy Inc.,2880113,ELA,39,39,100.0,39,*,...,22,56.4,*,*,*,*,488,2016-17,None,488
3,Achievement First Hartford Academy Inc. District,2880013,Achievement First Hartford Academy Inc.,2880113,Math,39,39,100.0,38,*,...,13,34.2,*,*,*,*,494,2016-17,None,494
4,Amistad Academy District,2790013,Amistad Academy,2795113,ELA,50,48,96.0,47,*,...,33,70.2,*,*,*,*,511,2016-17,None,511
5,Amistad Academy District,2790013,Amistad Academy,2795113,Math,50,48,96.0,47,10,...,9,19.1,0,0.0,9,19.1,467,2016-17,None,467
6,Ansonia School District,0020011,Ansonia High School,0026111,ELA,124,119,96.0,119,33,...,50,42.0,11,9.2,61,51.3,482,2016-17,H,482
7,Ansonia School District,0020011,Ansonia High School,0026111,Math,124,119,96.0,119,52,...,*,*,*,*,21,17.6,447,2016-17,H,447
8,Ansonia School District,0020011,P.A.C.E. (Positive And Creative Education),0020111,ELA,8,6,*,6,*,...,*,*,*,*,*,*,*,2016-17,H,*
9,Ansonia School District,0020011,P.A.C.E. (Positive And Creative Education),0020111,Math,8,6,*,6,*,...,*,*,*,*,*,*,*,2016-17,H,*
10,Area Cooperative Educational Services,2440014,Whitney High School North,2449114,ELA,18,15,*,15,*,...,*,*,*,*,*,*,*,2016-17,None,*
11,Area Cooperative Educational Services,2440014,Whitney High School North,2449114,Math,18,15,*,13,*,...,*,*,*,*,*,*,*,2016-17,None,*


In [47]:
get_df("all","2017").to_csv("../clean/for_db/sat_all_2017.csv",index=False)

In [48]:
get_df("race-eth","2017").to_csv("../clean/for_db/sat_race_2017.csv",index=False)

In [49]:
def weighted_avg(df, groups=[]):
    ret = df.copy()
    
    ret["level_3_4met_or_exceeded_averagescore"] = pd.to_numeric(ret["level_3_4met_or_exceeded_averagescore"],errors="coerce")
    ret["total_numberwith_scored_tests"] = pd.to_numeric(ret["total_numberwith_scored_tests"],errors="coerce")
    ret["weight"] = ret["level_3_4met_or_exceeded_averagescore"] * ret["total_numberwith_scored_tests"]
    
    
    ret = ret[ret["total_numberwith_scored_tests"].notnull()]
    ret = ret[ret["level_3_4met_or_exceeded_averagescore"].notnull()]

    ret = ret.groupby(["year","subject","drg"] + groups).agg({
            "weight":np.sum,
            "total_numberwith_scored_tests":np.sum
        }).reset_index()
    
    ret["average_score"] = ret["weight"] / ret["total_numberwith_scored_tests"]
    return ret
    
weighted_avg(get_df("all","2017"))

,year,subject,drg,total_numberwith_scored_tests,weight,average_score
0,2016-17,ELA,A,1900.0,1170275.0,615.934211
1,2016-17,ELA,B,5163.0,3003811.0,581.795661
2,2016-17,ELA,C,2132.0,1204510.0,564.967167
3,2016-17,ELA,D,3624.0,1974658.0,544.883554
4,2016-17,ELA,E,774.0,423896.0,547.669251
5,2016-17,ELA,F,1041.0,544595.0,523.146013
6,2016-17,ELA,G,4299.0,2141413.0,498.118865
7,2016-17,ELA,H,3835.0,1884301.0,491.343155
8,2016-17,ELA,I,5235.0,2367730.0,452.288443
9,2016-17,Math,A,1896.0,1155328.0,609.350211


In [50]:
weighted_avg(get_df("all","2017")).to_csv("../clean/for_db/drg_sat_all_2017.csv", index=False)

In [51]:
weighted_avg(get_df("race-eth","2017"),groups=["race"]).to_csv("../clean/for_db/drg_sat_race_2017.csv", index=False)

In [52]:
get_df("ell","2017")

,district,district_code,school,school_code,subject,ell,total_numberof_students,total_numbertested,connecticut_school_day_satparticipationrate,total_numberwith_scored_tests,...,level_3met_count,level_3met_pct,level_4exceeded_count,level_4exceeded_pct,level_3_4met_or_exceeded_count,level_3_4met_or_exceeded_pct,level_3_4met_or_exceeded_averagescore,year,drg,average_score
2,Achievement First Hartford Academy Inc. District,2880013,Achievement First Hartford Academy Inc.,2880113,ELA,N,39,39,100.0,39,...,22,56.4,*,*,*,*,488,2016-17,None,488
3,Achievement First Hartford Academy Inc. District,2880013,Achievement First Hartford Academy Inc.,2880113,Math,N,39,39,100.0,38,...,13,34.2,*,*,*,*,494,2016-17,None,494
4,Amistad Academy District,2790013,Amistad Academy,2795113,ELA,N,*,*,*,*,...,*,*,*,*,*,*,*,2016-17,None,*
5,Amistad Academy District,2790013,Amistad Academy,2795113,ELA,Y,*,*,*,*,...,*,*,*,*,*,*,*,2016-17,None,*
6,Amistad Academy District,2790013,Amistad Academy,2795113,Math,N,*,*,*,*,...,*,*,0,0.0,*,*,*,2016-17,None,*
7,Amistad Academy District,2790013,Amistad Academy,2795113,Math,Y,*,*,*,*,...,*,*,*,*,*,*,*,2016-17,None,*
8,Ansonia School District,0020011,Ansonia High School,0026111,ELA,N,*,*,*,*,...,*,*,*,*,*,*,*,2016-17,H,*
9,Ansonia School District,0020011,Ansonia High School,0026111,ELA,Y,*,*,*,*,...,*,*,*,*,*,*,*,2016-17,H,*
10,Ansonia School District,0020011,Ansonia High School,0026111,Math,N,*,*,*,*,...,*,*,*,*,*,*,*,2016-17,H,*
11,Ansonia School District,0020011,Ansonia High School,0026111,Math,Y,*,*,*,*,...,*,*,*,*,*,*,*,2016-17,H,*


In [53]:
weighted_avg(get_df("ell","2017"),groups=["ell"])#.to_csv("../clean/for_db/drg_sat_race.csv", index=False)

,year,subject,drg,ell,total_numberwith_scored_tests,weight,average_score
0,2016-17,ELA,A,N,325.0,203775.0,627.000000
1,2016-17,ELA,B,N,2619.0,1531617.0,584.809851
2,2016-17,ELA,C,N,1889.0,1073686.0,568.388565
3,2016-17,ELA,D,N,1921.0,1046015.0,544.515877
4,2016-17,ELA,E,N,1068.0,576254.0,539.563670
5,2016-17,ELA,F,N,514.0,265775.0,517.071984
6,2016-17,ELA,G,N,2816.0,1420994.0,504.614347
7,2016-17,ELA,G,Y,29.0,11832.0,408.000000
8,2016-17,ELA,H,N,3079.0,1557947.0,505.991231
9,2016-17,ELA,H,Y,434.0,165010.0,380.207373


In [54]:
def state_report(group="all",year="2017"):
    
    ret = pd.read_csv("../scripts/noequals/CTSchoolDaySAT-state-"+group+ "-" + year + ".csv",skiprows=2,header=None)
    ret = gs.merge_01(ret)
    ret = gs.clean_colnames(ret)
    
    ret["level"] = "Statewide"
    ret["year"] = fyear(year)
    
    ret["average_score"] = ret["level_3_4met_or_exceeded_averagescore"]
    
    return ret

state_report(group="race",year="2017")

,district,subject,race,total_numberof_students,total_numbertested,connecticut_school_day_satparticipationrate,total_numberwith_scored_tests,level_1not_met_count,level_1not_met_pct,level_2approaching_count,...,level_3met_count,level_3met_pct,level_4exceeded_count,level_4exceeded_pct,level_3_4met_or_exceeded_count,level_3_4met_or_exceeded_pct,level_3_4met_or_exceeded_averagescore,level,year,average_score
2,State of Connecticut,ELA,American Indian or Alaska Native,126,123,97.6,122,36,29.5,*,...,50,41.0,*,*,*,*,485,Statewide,2016-17,485
3,State of Connecticut,ELA,Asian,1975,1952,98.8,1952,158,8.1,237,...,847,43.4,710,36.4,1557,79.8,577,Statewide,2016-17,577
4,State of Connecticut,ELA,Black or African American,5179,4814,93.0,4814,1699,35.3,1344,...,1591,33.0,180,3.7,1771,36.8,456,Statewide,2016-17,456
5,State of Connecticut,ELA,Hispanic/Latino of any race,8122,7559,93.1,7558,2695,35.7,1869,...,2610,34.5,384,5.1,2994,39.6,460,Statewide,2016-17,460
6,State of Connecticut,ELA,Native Hawaiian or Other Pacific Islander,43,42,97.7,42,13,31.0,*,...,19,45.2,*,*,*,*,502,Statewide,2016-17,502
7,State of Connecticut,ELA,Two or More Races,825,796,96.5,796,110,13.8,142,...,385,48.4,159,20.0,544,68.3,529,Statewide,2016-17,529
8,State of Connecticut,ELA,White,23732,23135,97.5,23135,2056,8.9,2912,...,12276,53.1,5891,25.5,18167,78.5,555,Statewide,2016-17,555
9,State of Connecticut,Math,American Indian or Alaska Native,126,123,97.6,122,40,32.8,57,...,*,*,*,*,25,20.5,460,Statewide,2016-17,460
10,State of Connecticut,Math,Asian,1975,1952,98.8,1949,143,7.3,509,...,614,31.5,683,35.0,1297,66.5,588,Statewide,2016-17,588
11,State of Connecticut,Math,Black or African American,5179,4814,93.0,4789,2068,43.2,2122,...,528,11.0,71,1.5,599,12.5,432,Statewide,2016-17,432


In [55]:
state_report().to_csv("../clean/for_db/state_sat_all_2017.csv",index=False)

In [56]:
state_report("race").to_csv("../clean/for_db/state_sat_race_2017.csv",index=False)

In [57]:
state_report("ell").to_csv("../clean/for_db/state_sat_ell_2017.csv",index=False)